In [1]:
'''
tf.histogram_summary--记录张量监控数据随迭代进行的变化趋势[HISTOGRAMS][tf.summary.histogram]
tf.scalar_summary-----记录标量监控数据随迭代进行的变化趋势[EVENTS][tf.summary.scalar]
tf.image_summary------记录图片数据[IMAGES][tf.summary.image]
tf.audio_summary------记录音频数据[AUDIO][tf.summary.audio]
'''
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

SUMMARY_DIR = 'log/vomi'
BATCH_SIZE = 100
TRAIN_STEPS = 10000

# 变量监控
def variable_summaries(var, name):
    with tf.name_scope('summaries'):
        # 记录var所指的张量取值
        tf.summary.histogram(name, var)
        # 记录var平均值
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean/'+name, mean)
        # 记录var标准差
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev/'+name, stddev)

# 生成一层全连接层神经网络
def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    # 规定同一层网络在一个命名空间下
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            weights = tf.Variable(tf.truncated_normal([input_dim, output_dim], stddev=0.1))
            variable_summaries(weights, layer_name+'/weights')
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.constant(0.0, shape=[output_dim]))
            variable_summaries(biases, layer_name+'/biases')
        # 记录使用激活函数前的输出节点分布
        with tf.name_scope('Wx_plus_b'):
            preactivate = tf.matmul(input_tensor, weights) + biases
            tf.summary.histogram(layer_name+'/pre_activations', preactivate)
        # 记录使用激活函数后的输出节点分布
        activations = act(preactivate, name='activation')
        tf.summary.histogram(layer_name+'/activations', activations)
        # 返回
        return activations
    
def main(_):
    mnist = input_data.read_data_sets("MNIST_DATA", one_hot=True)
    # 定义输入
    with tf.name_scope('input'):
        x = tf.placeholder(tf.float32, [None, 784], name='x-input')
        y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')
    # 将输入向量还原成图片矩阵
    # tf.image_summary将图片信息写入日志[old version]
    with tf.name_scope('input_reshape'):
        image_reshaped_input = tf.reshape(x, [-1, 28, 28, 1])
        tf.summary.image('input', image_reshaped_input, 10)
    
    # 前向传播
    hidden1 = nn_layer(x, 784, 500, 'layer1')# 第一层使用默认ReLu激活函数
    y = nn_layer(hidden1, 500, 10, 'layer2', act=tf.identity)# 第二层tf.identity相当于x=y即未使用激活函数
    
    # 计算交叉熵，并记录日志
    with tf.name_scope('cross_entropy'):
        cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.argmax(y_,1), logits=y))
        tf.summary.scalar('cross_entropy', cross_entropy)
    
    # 反向传播
    with tf.name_scope('train'):
        train_step = tf.train.GradientDescentOptimizer(0.001).minimize(cross_entropy)
        
    # 正确率
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('accuracy', accuracy)
    
    # 执行所有日志生成操作
    #f.merge_all_summaries()[old version]
    merged = tf.summary.merge_all()
    
    with tf.Session() as sess:
        # 初始化写日志的writer，并写入计算图
        summary_writer = tf.summary.FileWriter(SUMMARY_DIR, sess.graph)
        tf.global_variables_initializer().run()
        for i in range(TRAIN_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            # 执行训练和日志生成操作，并返回日志
            summary, _ =sess.run([merged, train_step], feed_dict={x:xs, y_:ys})
            # 写入日志
            summary_writer.add_summary(summary, i)
    summary_writer.close()
    print('done')

if __name__ == '__main__':
    tf.app.run()

Extracting MNIST_DATA/train-images-idx3-ubyte.gz
Extracting MNIST_DATA/train-labels-idx1-ubyte.gz
Extracting MNIST_DATA/t10k-images-idx3-ubyte.gz
Extracting MNIST_DATA/t10k-labels-idx1-ubyte.gz
done


SystemExit: 

/home/fyzer/anaconda3/envs/tf/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
